In [4]:
from sqlalchemy import create_engine, text, inspect
import sqlite3
import os
import pandas as pd
!pip install gdown
import gdown

file_id = "1HE95jS3H9xWzmEJSD12QR5C6ufKkXj2E"
url = f"https://drive.google.com/uc?id={file_id}"
output = "creds.db"

gdown.download(url, output, quiet=False)

conn = sqlite3.connect(output)
cur = conn.cursor()

# get access parameters
cur.execute("SELECT * FROM access LIMIT 1;")
row = [r for r in cur.fetchall()]
cur.execute("PRAGMA table_info(access)")
cols = [r for r in cur.fetchall()]
conn.close()

# create dict with access parameters
for i in range(len(cols)):
    cols[i] = list(cols[i])
cols_transpose = [list(row) for row in zip(*cols)]
data = dict(zip(cols_transpose[1], row[0])) # учетные данные для подключения к бд


db_user = data['user']
db_password = data['pass']
db_url = data['url']
db_port = data['port']
db_root_base = 'homeworks'

file_id = '1PLRMh5K3zbbXbN8HYnyciJWkfhosadFs'
file_url = f"https://drive.google.com/uc?id={file_id}"
df = pd.read_csv(file_url, encoding="UTF-8")
df = df[:100]
# df["id"] = df.index

engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_url}:{db_port}/{db_root_base}",  # noqa
    pool_recycle=3600,
    # echo=True,
)

df.to_sql(
    name="topalova",
    con=engine,
    schema="public",
    if_exists="replace",  # или "append"
    index=True,
)

with engine.begin() as conn:
    rows = conn.execute(text('ALTER TABLE public.topalova ADD PRIMARY KEY (index)'))

inspector = inspect(engine)

columns = inspector.get_columns("topalova", schema="public")

{col["name"]: col["type"] for col in columns}


# with engine.begin() as conn:
#     query = """
#     SELECT t.country
#     FROM public.topalova AS t
#     LIMIT 10
#     """
#     rows = conn.execute(text(query)).all()

# rows

Downloading...
From: https://drive.google.com/uc?id=1HE95jS3H9xWzmEJSD12QR5C6ufKkXj2E
To: /content/creds.db
100%|██████████| 12.3k/12.3k [00:00<00:00, 29.5MB/s]


[('Jp',),
 ('Se',),
 ('Za',),
 ('Kr',),
 ('Au',),
 ('Ke',),
 ('Us',),
 ('Us',),
 ('Br',),
 ('Ar',)]